<a href="https://colab.research.google.com/github/karimamd/Bone-Abnormality-Classifier-in-Keras/blob/Khaled-Branch/marmora_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget -c https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip
!unzip MURA-v1.1.zip
!rm MURA-v1.1.zip

--2019-01-14 16:55:30--  https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3380245855 (3.1G) [application/zip]
Saving to: ‘MURA-v1.1.zip’

MURA-v1.1.zip         6%[>                   ] 208.01M  11.6MB/s    eta 4m 47s 

In [17]:
!pip uninstall keras
!pip3 install git+https://github.com/keras-team/keras
!pip uninstall keras-preprocessing
!pip3 install git+https://github.com/keras-team/keras-preprocessing

Uninstalling Keras-2.2.4:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.2.4.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.4
  Cloning https://github.com/keras-team/keras to /tmp/pip-req-build-dj7ox5i0
  Running setup.py bdist_wheel for Keras ... - \ | done
  Stored in directory: /tmp/pip-ephem-wheel-cache-yp9dtjx9/wheels/18/59/26/2a3c8d65212670e9526dcb6966eba15ee401e814aa74ca121d
Successfully built Keras
Uninstalling Keras-Preprocessing-1.0.5:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras_Preprocessing-1.0.5.dist-info/*
    /usr/local/lib/python3.6/dist-packages/keras_preprocessing/*
Proceed (y/n)? y
  Successfully uninstalled Keras-Preprocessing-1.0.5
 

In [1]:
!ls
# RESTART RUNTIME BEFORE PROCEEDING

Dropbox-Uploader     MURA-v1.1	    sample_data
dropbox_uploader.sh  MURA-v1.1.zip  token.txt


In [5]:
!wget https://www.dropbox.com/s/70a2a39hiix6anm/train_paths_labels.csv
!wget https://www.dropbox.com/s/8kfselmk1m3fphm/valid_paths_labels.csv

--2019-01-14 17:44:08--  https://www.dropbox.com/s/70a2a39hiix6anm/train_paths_labels.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/70a2a39hiix6anm/train_paths_labels.csv [following]
--2019-01-14 17:44:08--  https://www.dropbox.com/s/raw/70a2a39hiix6anm/train_paths_labels.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca98fbdf2e2103975f312404e86.dl.dropboxusercontent.com/cd/0/inline/AZaX1xJ0H1eDEt6Bj2lWM5SwEHQ5Zn8mQ78DHPZbJIQHo9hy3Hc-c3BL7-gdNS1VepP9VrfKRnFSivuYJ9RzWZlcSwO3WzQ_udbQUbdoyW53Z6s75xEqbCs__9n-yW-Z3I_fUUo-zLUa2TFe4IQCluVGTtiApcFWQ3xeyLMGrBo_AWSjIwabGo08nqUluwOb_jM/file [following]
--2019-01-14 17:44:09--  https://uca98fbdf2e2103975f312404e86.dl.dropboxusercontent.com/cd/0/inline/AZaX1xJ0H1eDEt6Bj2lWM

In [40]:
!rm -r Dropbox-Uploader
!rm dropbox_uploader.sh
!rm valid_paths_labels.csv
!rm train_paths_labels.csv

rm: cannot remove 'Dropbox-Uploader': No such file or directory
rm: cannot remove 'dropbox_uploader.sh': No such file or directory
rm: cannot remove 'valid_paths_labels.csv': No such file or directory
rm: cannot remove 'train_paths_labels.csv': No such file or directory


In [0]:
import pandas as pd
import numpy as np
import os, time, signal, shutil
import multiprocessing as mp
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import keras
from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.densenet import DenseNet169
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.nasnet import NASNetMobile
from keras.preprocessing import image
from keras.applications.mobilenetv2 import preprocess_input
from keras.applications import MobileNet
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard)
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.metrics import binary_accuracy, binary_crossentropy
from keras.models import Model, Sequential
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenetv2 import MobileNetV2
from keras.preprocessing import image as k_im_prep
from keras import backend as K
from keras_preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.utils import plot_model
from keras.layers.merge import concatenate


In [0]:
# from os import listdir
# from os.path import isfile, join
# onlyfiles = [f for f in listdir("MURA-v1.1")]
# print(onlyfiles)

## SETUP

In [29]:
!bash dropbox_uploader.sh download README.md

bash: dropbox_uploader.sh: No such file or directory


In [3]:
# just run the cell to be able to upload and download from Dropbox
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
!chmod +x Dropbox-Uploader/dropbox_uploader.sh
source = 'Dropbox-Uploader'
dest1 = '/content'
shutil.move(source+'/'+'dropbox_uploader.sh', dest1)
!echo "EAQdVV4B6swAAAAAAAAMNeyKygEAA5FP7QaiPVRRgJwupFyPIyq0DaAP6LiVy23c" > token.txt

fatal: destination path 'Dropbox-Uploader' already exists and is not an empty directory.
chmod: cannot access 'Dropbox-Uploader/dropbox_uploader.sh': No such file or directory


NameError: ignored

In [6]:
# download train and valid csv files that include images path and their labels
!bash dropbox_uploader.sh download train_paths_labels.csv
!bash dropbox_uploader.sh download valid_paths_labels.csv
# the saved model file
model_file= 'model.h5'

 > No such file or directory: /train_paths_labels.csv
Some error occured. Please check the log.
 > No such file or directory: /valid_paths_labels.csv
Some error occured. Please check the log.


In [0]:
# watch_ monitors the state of certain file if it is modified and uploads it to Dropbox if so
def watch_(file, interval):
  from datetime import datetime
  first_Time=False
  while True:
    if os.path.isfile(file):
      if not first_Time:
        os.system("bash dropbox_uploader.sh upload "+file+" "+file)
        first_Time=True
      moddate = os.stat(file)[8]
      time.sleep(interval)
      moddate_ = os.stat(file)[8]
      if moddate < moddate_:
        os.system("bash dropbox_uploader.sh upload "+file+" "+file)
    else:
      time.sleep(interval)
      

## MODEL

In [0]:
def make_FT_model(base=1, imagenet=True, freeze_all=True, add_denses=True):
  
  #weights of pretrained model
  if (imagenet==True):
    w='imagenet'
  else:
    w=None
  
  #default because refrenced before assignment error, just scroll down
  base_model = MobileNetV2(input_shape= (224, 224, 3),weights=w, include_top=False)
  feature_Concatenation_model1=NASNetMobile(input_shape= (224, 224, 3),weights=w, include_top=False)
  feature_Concatenation_model2=MobileNetV2(input_shape= (224, 224, 3),weights=w, include_top=False)
#   feature_Concatenation_Merged=None
  #initializing pretrained model
  if (base==0):
    base_model = MobileNetV2(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 1):
    base_model = DenseNet169(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 2):
    base_model = InceptionV3(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 3):
    base_model = ResNet50(input_shape= (224, 224, 3),weights=w, include_top=False)   
  elif (base == 4):
    base_model = NASNetMobile(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 5): #Feature concatenated
    base_model = feature_Concatenation_model1   
 
  if (freeze_all):
    #freeze layers of densenet
    for layer in base_model.layers:
      layer.trainable= False 
  
  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  

  if(add_denses):
    

    if(base ==5):  #Feature concatenated
      
      print("Adding bases")

      x1 = feature_Concatenation_model1.output
      x1 = GlobalAveragePooling2D()(x1)
   
      x2 = feature_Concatenation_model2.output
      x2 = GlobalAveragePooling2D()(x2)
      
      
      x1 = Dense(512, activation='relu')(x1)
      x1 = Dense(128, activation='relu')(x1)
      
      print("First Dense")
      
      x2 = Dense(512, activation='relu')(x2)
      x2 = Dense(128, activation='relu')(x2)
      
      print("Secomd Dense")

      merge = concatenate([x1, x2])
      
      print("Meging Dense")

      predictions = Dense(1, activation='sigmoid')(merge)
      
      print("Model Creation")

      feature_Concatenation_Merged = Model(inputs=[feature_Concatenation_model1.input,feature_Concatenation_model2.input],outputs=predictions)
      model= feature_Concatenation_Merged
      print("all here done")

      
    else:
      # let's add a fully-connected layer
      #x = Dense(1024, activation='relu')(x)
      x = Dense(512, activation='relu')(x)
      x = Dense(128, activation='relu')(x)
      #x = Dense(32, activation='relu')(x)
      # and a logistic layer -- let's say we have 200 classes

      predictions = Dense(1, activation='sigmoid')(x)

      # this is the model we will train
      model = Model(inputs=base_model.input, outputs=predictions)
    
  else:
    # just feature extractor
    model = Model(inputs=base_model.input, output=x)
  
  
  plot_model(model,to_file='demo.png',show_shapes=True)
  print(model.summary())

  return model

bug: if called fn on another base model it still loads the model

In [0]:
#KAREEM'S COPY OF FUNCTION JUST COMMENT THE CELL AND EVERYTHING IS NORMAL
def evaluate_limps(model=1,epoch=5,batch=32, imagenet=True, freeze_all=False,verbose=2):
  
  df_train=pd.read_csv('train_paths_labels.csv')
  df_valid=pd.read_csv('valid_paths_labels.csv')
  
  datagen = ImageDataGenerator(  rescale=1./255,
    featurewise_center=True,  #CHANGED IT TO TRUE # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  #CHANGED IT TO TRUE# divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
    zoom_range=0.1,
    channel_shift_range=0.,
    fill_mode='nearest')
  
  datagen.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
  datagen.std  = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
  train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=None,x_col="Img_Path",
                                              y_col="Label", class_mode="binary", target_size=(224,224), batch_size=batch)
  valid_generator=datagen.flow_from_dataframe(dataframe=df_valid, directory=None,x_col="Img_Path",
                                              y_col="Label", class_mode="binary", target_size=(224,224), batch_size=batch)
  print(len(train_generator))
  print("making model")
  if not os.path.isfile(model_file):
    model=make_FT_model(base= model, imagenet=imagenet, freeze_all=freeze_all, add_denses=True)
  else:
    model= load_model(model_file, compile=False)
  
  print("compiling")
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  
  checkpoint= ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
  callbacks_list = [checkpoint]
    
  step_train=train_generator.n//train_generator.batch_size
  step_valid=valid_generator.n//valid_generator.batch_size
  model.fit_generator(generator=train_generator, steps_per_epoch=step_train, epochs=epoch,
                      validation_data=valid_generator, validation_steps=step_valid, shuffle=True,
                      verbose=verbose, callbacks=callbacks_list)
   #
  
  
  loss_tr, accuracy_tr =model.evaluate_generator(train_generator, use_multiprocessing=True,steps=step_train)
  print("training loss/accuracy: ", loss_tr,'/', accuracy_tr)

  loss_val, accuracy_val = model.evaluate_generator(valid_generator, use_multiprocessing=True,steps=step_valid)
  print("validation loss/accuracy: ", loss_val,'/', accuracy_val)
    
    

In [9]:
!rm model.h5

rm: cannot remove 'model.h5': No such file or directory


In [10]:
!ls

Dropbox-Uploader     MURA-v1.1	    sample_data  train_paths_labels.csv
dropbox_uploader.sh  MURA-v1.1.zip  token.txt	 valid_paths_labels.csv


In [36]:
p = mp.Process(target=watch_, args=(model_file,5))
p.start()
evaluate_limps(model=5,epoch=1,batch=32,imagenet=True,freeze_all=False,verbose=1)


Found 36808 images belonging to 2 classes.
Found 3197 images belonging to 2 classes.
1151
making model
Adding bases
First Dense
Secomd Dense
Meging Dense
Model Creation


ValueError: ignored